In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")


In [ ]:
df = pd.read_csv('Social_Network_Ads.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df.Purchased.value_counts()

In [ ]:
df.Gender.value_counts()

In [ ]:
sns.boxplot(y='Age', x='Purchased', data=df)

In [ ]:
sns.boxplot(y='EstimatedSalary', x='Purchased', data=df)

In [ ]:
plt.hist(x="Purchased", data=df);
plt.title('Distribution of Purchase');
plt.ylabel('Count');
plt.xlabel('Purchase');

In [ ]:
plt.figure(figsize=(20,5))
bins_size = np.arange(15000,150000+10000,1000)
plt.hist(x="EstimatedSalary", data=df, bins= bins_size,rwidth=0.9);
plt.title('Distribution of Salary');
plt.ylabel('Count');
plt.xlabel('Salary');

In [ ]:
df.EstimatedSalary.mean()

In [ ]:
plt.figure(figsize=(15,5))
bins_size = np.arange(18,65,2)
plt.hist(x="Age", data=df, bins= bins_size,rwidth=0.9);
plt.title('Distribution of Age');
plt.ylabel('Count');
plt.xlabel('Age');

In [ ]:
sns.distplot(df.Purchased);

In [ ]:
df.describe()

In [ ]:
df.Age.unique()

In [ ]:
df.Age.nunique()

In [ ]:
df.EstimatedSalary.unique()

In [ ]:
df.EstimatedSalary.nunique()

In [ ]:
df.Purchased.unique()

In [ ]:
df.duplicated().sum()

In [ ]:
# Remove non-numeric columns (e.g., 'Gender')
df1 = df.drop(columns=['Gender'])

# Now you can proceed with your analysis
plt.figure(figsize=(8, 8))
ax = sns.heatmap(df1.corr(), square=True, annot=True, cmap='Spectral')
ax.set_ylim(4.0, 0)
plt.show()

In [ ]:
ax = sns.regplot(x="EstimatedSalary", y="Purchased", data=df)

In [ ]:
ax = sns.regplot(x="Age", y="Purchased", data=df)

In [ ]:
col = sns.color_palette()[0]

In [ ]:
sns.barplot(x="Purchased", y="EstimatedSalary", data=df, color=col)

In [ ]:
sns.barplot(x="Purchased", y="EstimatedSalary", data=df, hue="Gender")

In [ ]:
sns.pairplot(df, vars=["Age", "EstimatedSalary","Purchased"])

In [ ]:
sns.pairplot(df, vars=["Age", "EstimatedSalary","Purchased"],hue = 'Gender')

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.drop(columns=['User ID'], inplace = True)

In [ ]:
df.loc[((df.Age >58) & (df.Purchased==0)), 'Age'] = np.nan
df.fillna(53,inplace=True)

In [ ]:
df.loc[(df.EstimatedSalary>120000) & (df.Purchased==0), 'EstimatedSalary'] = np.nan
df.fillna(120000,inplace=True)

In [ ]:
from scipy import stats
z = np.abs(stats.zscore(df['EstimatedSalary']))
threshold = 3
print(np.where(z > 3))

In [ ]:
z = np.abs(stats.zscore(df['Age']))
print(np.where(z > 3))

In [ ]:
df.Age = df.Age.astype("int64")

In [ ]:
df.EstimatedSalary = df.EstimatedSalary.astype("int64")

In [ ]:
df.info()

In [ ]:
a = df.groupby(['Gender', 'Age'])
a.first()

In [ ]:
a = df.groupby(['Purchased','EstimatedSalary'])
a.first()

In [ ]:
sns.scatterplot(y="EstimatedSalary", x="Purchased", data=df)

In [ ]:
plt.figure(figsize = (15,8))
sns.scatterplot(y="EstimatedSalary", x="Age", data=df, hue = 'Purchased')

In [ ]:
sns.catplot(y="EstimatedSalary", x="Purchased", data=df, hue = 'Gender')

In [ ]:
sns.catplot(y="Age", x="Purchased", data=df, hue = 'Gender')

In [ ]:
df.Gender.replace({'Male':1,
                   'Female':0}, inplace=True)

In [ ]:
df.head()

In [ ]:
X = df.iloc[:, [1, 2]]
y = df.iloc[:, 3]

In [ ]:
X

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train,y_train)
knn_pred = knn.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,knn_pred))
print(classification_report(y_test,knn_pred))

In [ ]:
error_rate = []

for i in range(1,40):
    
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train,y_train)
    pred_i = knn.predict(X_test)
    error_rate.append(np.mean(pred_i != y_test))

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(range(1,40),error_rate,color='blue', linestyle='dashed', marker='o',
         markerfacecolor='red', markersize=10)
plt.title('Error Rate vs. K Value')
plt.xlabel('K')
plt.ylabel('Error Rate')

In [ ]:
knn = KNeighborsClassifier(n_neighbors=4)

knn.fit(X_train,y_train)
knn_pred = knn.predict(X_test)

print('WITH K=4')
print('\n')
print(confusion_matrix(y_test,knn_pred))
print('\n')
print(classification_report(y_test,knn_pred))

In [ ]:
from sklearn.metrics import accuracy_score
print ('accuracy_score : ', accuracy_score(y_test,knn_pred))

In [ ]:
from sklearn.model_selection import cross_val_score
knn_accuracy = cross_val_score(knn,X,y, cv = 5)

In [ ]:
knn_accuracy

In [ ]:
knn_accuracy.mean()

In [ ]:
from sklearn.metrics import roc_curve, auc

knn_fpr, knn_tpr, threshold = roc_curve(y_test, knn_pred)
auc_knn = auc(knn_fpr, knn_tpr)

plt.figure(figsize=(5, 5), dpi=100)
plt.plot(knn_fpr, knn_tpr, marker='.', label='Knn (auc = %0.3f)' % auc_knn)

plt.xlabel('False Positive Rate -->')
plt.ylabel('True Positive Rate -->')

plt.legend()

plt.show()

In [ ]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(X_train,y_train)
log_pred = log_reg.predict(X_test)

In [ ]:
print(confusion_matrix(y_test,log_pred))
print('\n')
print(classification_report(y_test,log_pred))

In [ ]:
log_accuracy = cross_val_score(log_reg,X,y, cv = 5)
print(log_accuracy)
print("mean value of accuracy",log_accuracy.mean())

In [ ]:
from sklearn.svm import SVC
svc_classifier = SVC(kernel = 'rbf', random_state = 0)
svc_classifier.fit(X_train, y_train)
svc_pred = svc_classifier.predict(X_test)

In [ ]:
print(confusion_matrix(y_test,svc_pred))
print('\n')
print(classification_report(y_test,svc_pred))

In [ ]:
svc_accuracy = cross_val_score(svc_classifier,X,y, cv = 5)
print(svc_accuracy)
print("mean value of accuracy",svc_accuracy.mean())

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt_classifier = DecisionTreeClassifier(criterion='entropy', random_state=0)
dt_classifier.fit(X_train, y_train)
dt_pred = dt_classifier.predict(X_test)

In [ ]:
print(confusion_matrix(y_test,dt_pred))
print('\n')
print(classification_report(y_test,dt_pred))

In [ ]:
dt_accuracy = cross_val_score(dt_classifier,X,y, cv = 5)
print(dt_accuracy)
print("mean value of accuracy",dt_accuracy.mean())

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
rf_classifier.fit(X_train, y_train)
rf_pred = rf_classifier.predict(X_test)

In [ ]:
print(confusion_matrix(y_test,rf_pred))
print('\n')
print(classification_report(y_test,rf_pred))

In [ ]:
rf_accuracy = cross_val_score(rf_classifier,X,y, cv = 5)
print(rf_accuracy)
print("mean value of accuracy",rf_accuracy.mean())

In [ ]:
print("For Random Forest Classifier::")
print(confusion_matrix(y_test,rf_pred))
print('\n')
print(classification_report(y_test,rf_pred))

In [ ]:

from sklearn.model_selection import GridSearchCV
parameters = [{"C": [1, 10, 100, 1000], "kernel": ['linear']}, 
              {"C": [1, 10, 100, 1000], "kernel": ['rbf'], 'gamma': [0.5, 0.1, 0.01, 0.001]}]

#Use this list to train
grid_search = GridSearchCV(estimator = svc_classifier, param_grid = parameters, scoring = 'accuracy', cv = 10, n_jobs = -1)
grid_search = grid_search.fit(X_train, y_train)

#Use attributes of grid_search to get the results
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

print("Best accuracy: ",best_accuracy)
print(best_parameters)

In [ ]:
from sklearn.svm import SVC
svc_classifier = SVC(kernel = 'rbf', random_state = 0, C =10, gamma=0.1)
svc_classifier.fit(X_train, y_train)
svc_pred = svc_classifier.predict(X_test)

In [ ]:
print("For SVM Classifier::")
print(confusion_matrix(y_test,svc_pred))
print('\n')
print(classification_report(y_test,svc_pred))

In [ ]:
n_estimators = [100, 300, 500]
max_depth = [5, 8, 15]
min_samples_leaf = [1, 2] 

hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,min_samples_leaf = min_samples_leaf)

gridF = GridSearchCV(rf_classifier, hyperF, cv = 5, verbose = 1, 
                      n_jobs = -1)
bestF = gridF.fit(X_train, y_train)

In [ ]:
bestF.best_estimator_

In [ ]:
bestF.best_params_

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 0, max_depth=5,min_samples_leaf=1)
rf_classifier.fit(X_train, y_train)
rf_pred = rf_classifier.predict(X_test)

In [ ]:
print(confusion_matrix(y_test,rf_pred))
print('\n')
print(classification_report(y_test,rf_pred))

In [ ]:
criterion = ['gini', 'entropy']
max_depth = [4,6,8,12]

parameters = dict(criterion=criterion,max_depth=max_depth)

  
clf = GridSearchCV(rf_classifier, hyperF, cv = 5, verbose = 1, n_jobs = -1)

# Fit the grid search
clf.fit(X_train, y_train)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt_classifier = DecisionTreeClassifier(criterion='entropy', random_state=0, max_depth=5)
dt_classifier.fit(X_train, y_train)
dt_pred = dt_classifier.predict(X_test)

In [ ]:
print(confusion_matrix(y_test,dt_pred))
print('\n')
print(classification_report(y_test,dt_pred))

In [ ]:
param_grid = [    
    {'solver' : ['lbfgs','newton-cg','liblinear','sag','saga'],
    'max_iter' : [100, 1000,2500, 5000]
    }
]
clf = GridSearchCV(log_reg, param_grid = param_grid, cv = 3, verbose=True, n_jobs=-1)
best_clf = clf.fit(X_train, y_train)

In [ ]:
a = best_clf.best_estimator_
a

In [ ]:
best_clf.best_params_

In [ ]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(C=1.0, solver='lbfgs',max_iter=100 )
log_reg.fit(X_train,y_train)
log_pred = log_reg.predict(X_test)

In [ ]:
print(confusion_matrix(y_test,log_pred))
print('\n')
print(classification_report(y_test,log_pred))

In [ ]:
leaf_size = list(range(1,10))
n_neighbors = list(range(1,10))
p=[1,2]

#Convert to dictionary
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)

#Use GridSearch
clf = GridSearchCV(knn, hyperparameters, cv=10, verbose=True, n_jobs=-1)
#Fit the model
best_model = clf.fit(X_train,y_train)

In [ ]:
best_model.best_estimator_

In [ ]:
best_model.best_params_

In [ ]:
knn = KNeighborsClassifier(n_neighbors=9, p = 1, leaf_size=1)

knn.fit(X_train,y_train)
knn_pred = knn.predict(X_test)

print(confusion_matrix(y_test,knn_pred))
print('\n')
print(classification_report(y_test,knn_pred))

In [ ]:
print(confusion_matrix(y_test,dt_pred))
print('\n')
print(classification_report(y_test,dt_pred))

In [ ]:
df.head()

In [ ]:
X1 = df.iloc[:,0: 3]
y1 = df.iloc[:, 3]

In [ ]:
X1

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size = 0.30, random_state = 0)

In [ ]:
dt_classifier = DecisionTreeClassifier(criterion='entropy', random_state=0, max_depth=5)
dt_classifier.fit(X_train, y_train)
dt_pred = dt_classifier.predict(X_test)

In [ ]:
print(confusion_matrix(y_test,dt_pred))
print('\n')
print(classification_report(y_test,dt_pred))